<img align="left" src="https://panoptes-uploads.zooniverse.org/project_avatar/86c23ca7-bbaa-4e84-8d8a-876819551431.png" type="image/png" height=100 width=100>
</img>


<h1 align="right">KSO Tutorials #12: Analyse Zooniverse classifications</h1>
<h3 align="right">Written by @jannesgg and @vykanton</h3>
<h5 align="right">Last updated: Nov 25th, 2021</h5>

In [ ]:
%load_ext autoreload
%autoreload 2

# Set up and requirements

### Import Python packages

In [ ]:
# Set the directory of the libraries
import sys
sys.path.append('..')

# Set to display dataframes as interactive tables
from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)

# Import required modules
import utils.tutorials_utils as t_utils
import utils.t12_utils as t12
import utils.zooniverse_utils as zoo

print("Packages loaded successfully")

### Choose your project

In [ ]:
project = t_utils.choose_project()

### Populate SQL database with sites, movies and species and connect to Zoo

In [ ]:
# Initiate db
db_info_dict = t_utils.initiate_db(project.value)

In [ ]:
# Connect to Zooniverse project
zoo_project = t_utils.connect_zoo_project(project.value)

### Retrieve Zooniverse information

In [ ]:
# Specify the Zooniverse information required throughout the tutorial
zoo_info = ["subjects", "workflows", "classifications"]

zoo_info_dict = t_utils.retrieve__populate_zoo_info(project_name = project.value, 
                           db_info_dict = db_info_dict,
                           zoo_project = zoo_project,
                           zoo_info = zoo_info)   

### Step 1: Specify the Zooniverse workflow id and version of interest

*Note:  A manual export in Zooniverse is required to get the most up-to-date classifications here**

Make sure your workflows in Zooniverse have different names to avoid issues while selecting the workflow id

In [ ]:
# Display a selectable list of workflow names and the type of subject
workflow_name, subj_type = t_utils.choose_single_workflow(zoo_info_dict["workflows"])

In [ ]:
# Display a selectable list of versions of the workflow of interest
workflow_version, workflow_id = t12.choose_w_version(zoo_info_dict["workflows"], workflow_name.value)

In [ ]:
# Retrieve classifications from the workflow of interest
class_df = t12.get_classifications(workflow_id = workflow_id,
                                   workflow_version = workflow_version.value, 
                                   subj_type = subj_type.value, 
                                   class_df = zoo_info_dict["classifications"],
                                   project_name = project.value,
                                  db_info_dict = db_info_dict)


### Step 2: Aggregate classifications received on the workflow of interest

In [ ]:
# Specify the agreement threshold required among cit scientists
agg_params = t12.choose_agg_parameters(subj_type.value)

In [ ]:
agg_class_df, raw_class_df = t12.aggregrate_classifications(class_df, subj_type.value, project.value, agg_params)

### Step 3: Summarise the number of classifications based on the agreement specified

In [ ]:
agg_class_df.groupby("label")["subject_ids"].agg("count")

### Step 4: Display the aggregated classifications in a table

In [ ]:
# Display the dataframe into a table
if subj_type.value == "clip":
    a = agg_class_df[["subject_ids","label","how_many","first_seen"]]
else:
    a = agg_class_df
a

### Step 5: Use the subject explorer widget to visualise subjects and their aggregated classifications

In [ ]:
# Launch the subject viewer
t12.launch_viewer(agg_class_df, subj_type.value)

### Step 6: Use the subject explorer widget to get more information about specific subjects and their "raw" classifications

In [ ]:
# Launch the classifications_per_subject explorer
t12.explore_classifications_per_subject(raw_class_df, subj_type.value)

In [ ]:
# END